In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")  # Replace with your key

from openai import OpenAI
client = OpenAI()

In [2]:
file = client.files.create(
  file=open("wiki.txt", "rb"),
  purpose='assistants'
)

# Add the file to the assistant
assistant = client.beta.assistants.create(
  instructions='''You are a code helper. Your role is now focused solely on helping users with terminal commands.
  Provide guidance on using various shell commands, explain command line interfaces, and assist in troubleshooting terminal issues.
  Offer clear, step-by-step instructions for executing and understanding terminal commands. Avoid providing commands that could be harmful or irreversible without proper warning.
  If unclear about the user's requirements, ask for clarification.
  Your responses should be informative, precise, and tailored to users ranging from beginners to advanced.''',
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

In [41]:
assistant

Assistant(id='asst_tkYkle3CFzT2ISdEoXZgIYM0', created_at=1706019403, description=None, file_ids=['file-fD1cNVK0qNlB5h5aoBYndV53'], instructions="You are a code helper. Your role is now focused solely on helping users with terminal commands.\n  Provide guidance on using various shell commands, explain command line interfaces, and assist in troubleshooting terminal issues.\n  Offer clear, step-by-step instructions for executing and understanding terminal commands. Avoid providing commands that could be harmful or irreversible without proper warning.\n  If unclear about the user's requirements, ask for clarification.\n  Your responses should be informative, precise, and tailored to users ranging from beginners to advanced.", metadata={}, model='gpt-4-1106-preview', name=None, object='assistant', tools=[ToolRetrieval(type='retrieval')])

In [3]:
thread = client.beta.threads.create()

In [27]:
# message = client.beta.threads.messages.create(
#     thread_id=thread.id,
#     role="user",
#     content="How to create an ec2 instance in eu-west-2?"
# )

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Okay cool, Im facing some error.. can you help?"
)

In [34]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_aEGtxI68Dx1SWJmQSH2vIAi7', assistant_id='asst_tkYkle3CFzT2ISdEoXZgIYM0', content=[MessageContentText(text=Text(annotations=[], value="Absolutely, I'd be happy to help you troubleshoot the error you're facing. Please provide me with the following information:\n\n1. The specific command or action you were trying to perform when you encountered the error.\n2. The exact text of the error message you received.\n3. Any relevant details about your AWS configuration or the state of your resources that might help diagnose the issue.\n\nRemember to omit any sensitive information such as your Access Key, Secret Key, or other credentials when sharing error messages."), type='text')], created_at=1706019640, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_CZi8VdMhi9a7DjcD2J8mrKov', thread_id='thread_lDjnYqjALmGzJYaN8Qpx7mhK'), ThreadMessage(id='msg_f0L9qoeYIayjaf52fFIZMjpw', assistant_id=None, content=[Message

In [35]:
for message in messages.data:
    text = message.content[0].text.value
    print('-' * 50)
    print(text)

--------------------------------------------------
Absolutely, I'd be happy to help you troubleshoot the error you're facing. Please provide me with the following information:

1. The specific command or action you were trying to perform when you encountered the error.
2. The exact text of the error message you received.
3. Any relevant details about your AWS configuration or the state of your resources that might help diagnose the issue.

Remember to omit any sensitive information such as your Access Key, Secret Key, or other credentials when sharing error messages.
--------------------------------------------------
Okay cool, Im facing some error.. can you help?
--------------------------------------------------
Creating an Amazon EC2 instance in the `eu-west-2` region (London) can be done using the AWS Management Console, AWS Command Line Interface (CLI), or AWS SDKs. Below I'll provide an example of how to do it using the AWS CLI.

Before you proceed, ensure you have the following:

parse messages

In [28]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [31]:
run

Run(id='run_CZi8VdMhi9a7DjcD2J8mrKov', assistant_id='asst_tkYkle3CFzT2ISdEoXZgIYM0', cancelled_at=None, completed_at=None, created_at=1706019639, expires_at=1706020239, failed_at=None, file_ids=['file-fD1cNVK0qNlB5h5aoBYndV53'], instructions="You are a code helper. Your role is now focused solely on helping users with terminal commands.\n  Provide guidance on using various shell commands, explain command line interfaces, and assist in troubleshooting terminal issues.\n  Offer clear, step-by-step instructions for executing and understanding terminal commands. Avoid providing commands that could be harmful or irreversible without proper warning.\n  If unclear about the user's requirements, ask for clarification.\n  Your responses should be informative, precise, and tailored to users ranging from beginners to advanced.", last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_lDjnYqjALmGzJYaN8

In [37]:
client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
).status

'completed'